<a href="https://colab.research.google.com/github/fortybyte/Auto-GPT/blob/master/clip_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install transformers datasets accelerate
!pip install pandas scikit-learn Pillow
!pip install git+https://github.com/openai/CLIP.git


  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-1y4qqsr1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-1y4qqsr1
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=21b568b9986933bce5ed20ba8ea59ef38bed983c88e3ea7f153aafb411554873
  Stored in directory: /tmp/pip-ephem-wheel-cache-87sx7wmv/wheels/3f/7c/a4/9b490845988bf7a4db33674d52f709f088f64392063872eb9a
Successfully built clip


In [ ]:
import os
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel

#paths
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
root_path="/content/drive/MyDrive/CLIPData/paint4poem_dataset_samples/Paint4Poem-Web-famous-subset"
csv_path = root_path + "/POEM_IMAGE.csv"
image_folder = root_path+"/images"
df = pd.read_csv(csv_path,delimiter="\t", header=None, names=["image_id", "poem"])


In [ ]:
model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [ ]:
import torch

def collate_fn(batch):
    # Extract the text token components from the batch
    input_ids = [item["input_ids"] for item in batch]
    attention_mask = [item["attention_mask"] for item in batch]

    # Use the processor's tokenizer to pad the text tokens
    padded_text = processor.tokenizer.pad(
        {"input_ids": input_ids, "attention_mask": attention_mask},
        return_tensors="pt"
    )

    # Stack image tensors (they should already be of fixed size)
    pixel_values = torch.stack([item["pixel_values"] for item in batch])

    # Combine the padded text tokens and the image tensors into one dictionary
    return {**padded_text, "pixel_values": pixel_values}


In [ ]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


class ClipDataset(Dataset):
    def __init__(self, dataframe, image_folder, processor):
        self.dataframe = dataframe.reset_index(drop=True)
        self.image_folder = image_folder
        self.processor = processor

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image_path = os.path.join(self.image_folder, row["image_id"])
        image_path = image_path + '.jpeg'
        image = Image.open(image_path).convert("RGB")
        text = row["poem"]
        inputs = self.processor(text=[text], images=[image], return_tensors="pt", padding=True)
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        return inputs



train_dataset = ClipDataset(train_df, image_folder, processor)
val_dataset = ClipDataset(val_df, image_folder, processor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True,collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=32,collate_fn=collate_fn)

In [ ]:
#freeze text
# for param in model.text_model.parameters():
#     param.requires_grad = False
# freeze image
# for param in model.vision_model.parameters():
#     param.requires_grad = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [ ]:
import torch.optim as optim

# Define hyperparameters (feel free to tweak)
learning_rate = 5e-5
num_epochs = 5  # adjust as needed

optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(**batch)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

model.eval()
with torch.no_grad():
    val_loss = 0.0
    for batch in val_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        val_loss += outputs.loss.item()
    avg_val_loss = val_loss / len(val_loader)
    print(f"Validation Loss: {avg_val_loss:.4f}")
model.save_pretrained("/content/drive/MyDrive/models")
processor.save_pretrained("/content/drive/MyDrive/models")


You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [ ]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import clip

# ------------------------------
# 1. Load CSV and Prepare Lists
# ------------------------------
# Update the CSV path and delimiter as needed
root_path="/content/drive/MyDrive/CLIPData/paint4poem_dataset_samples/Paint4Poem-Web-famous-subset"
csv_path = root_path + "/POEM_IMAGE.csv"
image_folder = root_path+"/images"
df = pd.read_csv(csv_path, delimiter="\t", header=0, names=["image_id", "poem"])

# Create lists of full image paths and corresponding poems
list_image_path = [os.path.join(image_folder, f"{image_id}.jpeg") for image_id in df["image_id"]]
list_txt = df["poem"].tolist()

# ------------------------------
# 2. Define the Dataset Class
# ------------------------------
class image_title_dataset(Dataset):
    def __init__(self, list_image_path, list_txt):
        # Initialize image paths and tokenize texts using CLIP's tokenizer with truncation enabled
        self.image_path = list_image_path
        self.title = clip.tokenize(list_txt, truncate=True)  # Automatically truncates texts beyond 77 tokens

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        # Preprocess image using CLIP's preprocessing function
        image = Image.open(self.image_path[idx]).convert("RGB")
        image = preprocess(image)
        title = self.title[idx]
        return image, title


# ------------------------------
# 3. Load CLIP Model and Preprocess Function
# ------------------------------
device = "cuda" if torch.cuda.is_available() else "cpu"
# Load the model and the preprocess function; using the ViT-B/32 variant
model, preprocess = clip.load("ViT-B/32", device=device)
model.to(device)

# ------------------------------
# 4. Create the DataLoader
# ------------------------------
# Instantiate the dataset and DataLoader
dataset = image_title_dataset(list_image_path, list_txt)
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# ------------------------------
# 5. Setup Optimizer and Loss Functions
# ------------------------------
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=5e-5,
    betas=(0.9, 0.98),
    eps=1e-6,
    weight_decay=0.2
)
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

# A helper function to convert model weights to fp32
def convert_models_to_fp32(model):
    for param in model.parameters():
        param.data = param.data.float()



  0%|          | 0/4 [00:00<?, ?it/s]


RuntimeError: Tensors of the same index must be on the same device and the same dtype except `step` tensors that can be CPU and float32/64 notwithstanding

In [ ]:
import torch.cuda.amp

scaler = torch.cuda.amp.GradScaler()

num_epochs = 30
for epoch in range(num_epochs):
    pbar = tqdm(train_dataloader, total=len(train_dataloader))
    for batch in pbar:
        optimizer.zero_grad()

        images, texts = batch
        images = images.to(device)
        texts = texts.to(device)

        with torch.cuda.amp.autocast():
            # Forward pass: CLIP returns logits for images and texts.
            logits_per_image, logits_per_text = model(images, texts)
            ground_truth = torch.arange(len(images), dtype=torch.long, device=device)
            total_loss = (loss_img(logits_per_image, ground_truth) + loss_txt(logits_per_text, ground_truth)) / 2

        # Backward pass with AMP scaling
        scaler.scale(total_loss).backward()
        scaler.step(optimizer)
        scaler.update()

        pbar.set_description(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss.item():.4f}")


<ipython-input-40-b2a975737c89>:3: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
  0%|          | 0/4 [00:00<?, ?it/s]<ipython-input-40-b2a975737c89>:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 30/30, Loss: 0.8154: 100%|██████████| 4/4 [00:01<00:00,  2.73it/s]


In [ ]:
def convert_models_to_fp32(model):
    for param in model.parameters():
        param.data = param.data.to(torch.float32)
        if param.grad is not None:
            param.grad.data = param.grad.data.to(torch.float32)
